In [6]:
import pandas as pd

In [7]:
indiv_2004 = pd.read_stata('./input/Individual_t104.dta', convert_categoricals= False) # Leo la base individual para 2004, no quiero que me guarde el formato de stata, así que no convierto las variables categóricas
#indiv_2024 = pd.read_excel('./input/usu_individual_T124.xlsx') # Leo la base individual para 2024, acá no tengo problemas con el formato
hog_2004 = pd.read_stata('./input/Hogar_t104.dta', convert_categoricals= False) # Leo la base de hogares para 2004, devuelta cambio el formato
#hog_2024 = pd.read_excel('./input/usu_hogar_T124.xlsx') # Leo la base de hogares para 2024

# Los excel tardan bastante en cargar, así que vamos a guardarlos en otro formato

In [8]:
#indiv_2024.to_csv('./input//usu_individual_T124.csv', index= False)
#hog_2024.to_csv('./input/usu_hogar_T124.csv', index = False)

In [9]:
tipos_indiv = indiv_2004.dtypes.to_dict() # Vemos los dtypes de la base de 2004
# Lo guardamos como diccionario para cuando cargue el csv poder especificarle al read_csv los dtypes de la base de 2024
# Cambiamos los dtypes de las columnas que nos van a a interesar
tipos_indiv['ch04'] = 'category'
tipos_indiv['estado'] = 'category'
tipos_indiv['cat_inac'] = 'category'
tipos_indiv['nivel_ed'] = 'category'
tipos_indiv['ch07'] = 'category'
tipos_indiv['ch08'] = 'category'
tipos_indiv['ano4'] = 'category'
tipos_indiv['aglomerado'] = 'category'
tipos_indiv['trimestre'] = 'category'
tipos_indiv['nro_hogar'] = 'category'

# Las asignamos al df
indiv_2004 = indiv_2004.astype(tipos_indiv)

In [16]:
# También guardo lod dtypes de la base de hogares para cargar bien el csv
tipos_hog = hog_2004.dtypes.to_dict()
tipos_hog['iv3'] = 'category'
tipos_hog['iv4'] = 'category'
tipos_hog['iv5'] = 'category'
tipos_hog['iv6'] = 'category'
tipos_hog['iv7'] = 'category'
tipos_hog['iv9'] = 'category'
tipos_hog['iv10'] = 'category'
tipos_hog['iv11'] = 'category'
tipos_hog['ii7'] = 'category'
tipos_hog['ii8'] = 'category'
tipos_hog['ii9'] = 'category'
tipos_hog['v1'] = 'category'
tipos_hog['v2'] = 'category'
tipos_hog['v21'] = 'category'
tipos_hog['v22'] = 'category'
tipos_hog['v3'] = 'category'
tipos_hog['v4'] = 'category'
tipos_hog['v5'] = 'category'
tipos_hog['v6'] = 'category'
tipos_hog['v7'] = 'category'
tipos_hog['v8'] = 'category'
tipos_hog['v9'] = 'category'
tipos_hog['v10'] = 'category'
tipos_hog['v11'] = 'category'
tipos_hog['v13'] = 'category'
tipos_hog['v14'] = 'category'
tipos_hog['v15'] = 'category'
tipos_hog['v16'] = 'category'
tipos_hog['v17'] = 'category'
tipos_hog['v18'] = 'category'
tipos_hog['v19_a'] = 'category'
tipos_hog['v19_b'] = 'category'

hog_2004 = hog_2004.astype(tipos_hog)

In [17]:
# Cargo los csv que guarde antes con el dtype correcto y más rápido que un excel
indiv_2024 = pd.read_csv('./input/usu_individual_T124.csv', dtype = tipos_indiv, low_memory= False)
hog_2024 = pd.read_csv('./input/usu_hogar_T124.csv', dtype = tipos_hog, low_memory= False)

In [18]:
cols_ind = indiv_2004.columns # columnas en la base individual
cols_hog = hog_2004.columns # columnas en la base de hogares

cols_comunes = [col for col in cols_hog if col in cols_ind] # guardo las columnas que tienen en común
data_2004 = pd.merge(indiv_2004, hog_2004, on= cols_comunes[:7], how='inner', suffixes=('_indiv', '_hog')) # hago el merge en base a las columnas que tienen en común
# Antes me duplicaba las columnas en el merge

In [19]:
# Las bases de 2024 tienen las variables en mayúscula, lo que después puede ser un problema
indiv_2024.columns = indiv_2024.columns.str.lower() # las paso a minúscula
hog_2024.columns = hog_2024.columns.str.lower()

# De vuelta agarro las columnas en común y hago el merge en base a esas
cols_ind = indiv_2024.columns
cols_hog = hog_2024.columns

cols_comunes = [col for col in cols_hog if col in cols_ind]
data_2024 = pd.merge(indiv_2024, hog_2024, on = cols_comunes[:7], how='inner', suffixes= ('_indiv', '_hog'))

# La unica variable que quiero que tenga nombre en mayúscula es CODUSU
data_2024.rename(columns={'codusu': 'CODUSU'}, inplace=True)

In [20]:
# Juntamos ambas bases en una única base
data = pd.concat([data_2004, data_2024], ignore_index= True)

In [22]:
# Agarro los dtypes correctos y se los aplico a la base entera
categorical_columns = indiv_2004.select_dtypes(include= 'category').columns
# Para compatibilizar ambas bases hacemos lo mismo que para la de 2004
for col in categorical_columns:
    data[col] = data[col].astype(int) # Las paso a integer

for col in categorical_columns:
    data[col] = data[col].astype('category') # Devuelta las paso a categoría

# Hago lo mismo para los dtypes de la de hogares
categorical_columns = hog_2004.select_dtypes(include = 'category')
for col in categorical_columns:
    data[col] = data[col].astype(int) # Las paso a integer

for col in categorical_columns:
    data[col] = data[col].astype('category') # Devuelta las paso a categoría


In [23]:
data = data.loc[data['aglomerado'] == 4] # Nos quedamos únicamente con las observaciones de Bahía Blanca

# Limpiamos la base. Nos quedamos con las observaciones que tienen un ingreso mayor o igual a 0, las edades mayores o iguales a 0
# Y nos quedamos con las observaciones que tienen el mismo ipcf para el individuo y para el hogar
data = data.loc[(data['ipcf_indiv']>=0) & (data['ch06']>=0) & (data['ipcf_hog'] == data['ipcf_indiv'])]

In [24]:
# Guardo el df en formato pickle para que me guarde los dtypes
data.to_pickle('./input/data.pkl')

In [25]:
# Cargo la base
data = pd.read_pickle('./input/data.pkl')